In [ ]:
import pandas as pd
import numpy as np
import math
import pylab
from pylab import *
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from IPython.display import display
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

import os

In [ ]:
save_folder = './Main_Figures/'
dataset = 'Pancreas'

In [ ]:
excel_path = '~/Downloads/All_metrics_15_Sep.xlsx' 
sheet_name =  'all_metrics_revision'

In [ ]:
columns_to_scale = ["NMI cluster/label","ARI cluster/label","ASW label","ASW label/batch","PCR batch","isolated f1 score","isolated silhouette coefficient","graph connectivity","kBET","iLISI","cLISI"]
method_color_dct = {'scVI' : '#28DDED', 'Harmony': '#ED7A28','Seurat' : '#994363', 
       'BBKNN': '#B626D3', 'Scanorama': '#EDBF28', 
       'INSCT' : '#286CED', 'LIGER' : '#90EE90','fastMNN':  "#FFB6C1", "iMAP" : "#964B00",
       'scDREAMER': '#086E28', 'scANVI': '#c5b0d5',
'scGEN': '#d62829',
'scDREAMER-Sup': '#113f0a'
       }
methods_to_plot = [ 'scVI' , 'Harmony','Seurat', 'BBKNN', 'Scanorama','INSCT' ,'LIGER' , "iMAP" ,'scDREAMER']

In [ ]:
import os
def plot_bar(df_, col_name,save_folder = False):#save_name dataset
    
    rc('axes', linewidth = 2)

#     if (dataset == 'Human_Mouse' and col_name == 'kBET'):
#         df_ = df_[df_.index != 'BBKNN']#.reset_index(drop = True)
#     if (dataset =='Human_Mouse' and col_name in ["isolated f1 score","isolated silhouette coefficient"]):
#         return
#     if (col_name == 'PCR batch'): # or col_name == 'ASW label' or col_name == 'ASW label/batch' or  col_name == 'isolated silhouette coefficient'
#         df_ = df_[df_.index != 'BBKNN']#.reset_index(drop = True)


    fig = plt.figure(figsize = (6, 4))
    ax = df_[col_name].plot(kind="bar", color = df_['color'])

   
    rects = ax.patches
    ax.set_xticklabels(df_.index, rotation = 75, fontname='Arial', fontsize = 10)
  
    for rect, label in zip(rects, df_[col_name]):
        height = rect.get_height()
        ax.text(
        rect.get_x() + rect.get_width() / 2, height, round(label, 2), ha="center", va="bottom"
        )
    
    
    mi = df_[col_name].min()
    mx = df_[col_name].max()
    ylim(max(mi - 0.01,0), min(mx*1.05, 1.0))
    
    
    
    plt.ylabel(col_name, fontsize = 15, fontname='Arial', fontweight = 'bold')
    plt.xlabel(None)#, fontsize = 15, fontname='Arial', fontweight = 'bold')
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(14)
        tick.label1.set_fontweight('bold')
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(14)
        tick.label1.set_fontweight('bold')
        
    plt.subplot(111).spines['right'].set_visible(False)
    plt.subplot(111).spines['top'].set_visible(False)
   
    plt.tight_layout()
#     plt.gcf().subplots_adjust(bottom = 0.3)
    #plt.gcf().subplots_adjust(top = 3)
    #plt.gcf().subplots_adjust(top=5)
    
    if save_folder:
        if not os.path.exists(save_folder):
            print(save_folder)
            os.makedirs(save_folder)
        plt.savefig(save_folder+'/'+col_name.replace('/','_')+'.png', transparent=True, bbox_inches='tight')

    plt.show()

# composite score main

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def scale(df):
    scaler = MinMaxScaler()
    df = scaler.fit_transform(df)
    return df
def calculate_composite(df_dataset):
    df_dataset['color'] = pd.Series(df_dataset.index,index=df_dataset.index).replace(method_color_dct)
    df_dataset['Composite bio-conservation score'] = df_dataset[['NMI cluster/label', 'ARI cluster/label', 'ASW label']].mean(axis = 1)
    df_dataset['Composite batch-correction score'] = df_dataset[['ASW label/batch', 'PCR batch', 'graph connectivity', 'kBET']].mean(axis = 1)
    df_dataset['Composite isolated label score'] = df_dataset[['isolated silhouette coefficient', 'isolated f1 score']].mean(axis = 1)
    df_dataset['Combined composite score'] = df_dataset[['Composite bio-conservation score', 'Composite batch-correction score']].mean(axis = 1)
    df_dataset = df_dataset.apply(lambda x:x.apply(lambda y:round(y,2) if type(y)!=str else y))
    return df_dataset
def plot_composite(df_dataset,save_path):
    plot_bar(df_dataset, 'Combined composite score', save_folder=save_path)
    plot_bar(df_dataset, 'Composite batch-correction score', save_folder = save_path)
    plot_bar(df_dataset, 'Composite bio-conservation score', save_folder=save_path)
    plot_bar(df_dataset, 'Composite isolated label score', save_folder=save_path)

# Pancreas

In [ ]:
dataset = 'Pancreas'
df = pd.read_excel(excel_path,sheet_name =sheet_name)
df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
df_dataset.index = df_dataset['Method']
df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
df_dataset  = df_dataset.loc[methods_to_plot,:]
df_dataset.loc[:,columns_to_scale] =scale(df_dataset.loc[:,columns_to_scale])
df_dataset = calculate_composite(df_dataset)
display(df_dataset)
plot_composite(df_dataset,save_folder+dataset)

In [ ]:
dataset = 'Pancreas'
df = pd.read_excel(excel_path,sheet_name =sheet_name)
df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
df_dataset.index = df_dataset['Method']
df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
df_dataset  = df_dataset.loc[methods_to_plot,:]
df_dataset.loc[:,columns_to_scale] =scale(df_dataset.loc[:,columns_to_scale])
df_dataset = calculate_composite(df_dataset)
df_dataset = df_dataset[["Composite bio-conservation score", "Composite batch-correction score",
                         "Composite isolated label score", "Combined composite score"]]
df_dataset.to_csv('./Composite_Scores/'+dataset+'_composite_scores.csv')

# Human_Retina

In [ ]:
dataset = "Human_Retina"
df = pd.read_excel(excel_path,sheet_name =sheet_name)
df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
df_dataset.index = df_dataset['Method']
df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
df_dataset  = df_dataset.loc[methods_to_plot,:]
df_dataset.loc[:,columns_to_scale] =scale(df_dataset.loc[:,columns_to_scale])
df_dataset = calculate_composite(df_dataset)
display(df_dataset)
plot_composite(df_dataset,save_folder+dataset)

In [ ]:
dataset = 'Human_Retina'
df = pd.read_excel(excel_path,sheet_name =sheet_name)
df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
df_dataset.index = df_dataset['Method']
df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
df_dataset  = df_dataset.loc[methods_to_plot,:]
df_dataset.loc[:,columns_to_scale] =scale(df_dataset.loc[:,columns_to_scale])
df_dataset = calculate_composite(df_dataset)
df_dataset = df_dataset[["Composite bio-conservation score", "Composite batch-correction score",
                         "Composite isolated label score", "Combined composite score"]]
df_dataset.to_csv('./Composite_Scores/'+dataset+'_composite_scores.csv')

# Human Mouse

In [ ]:
dataset = "Human_Mouse"
df = pd.read_excel(excel_path,sheet_name =sheet_name)
df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
df_dataset.index = df_dataset['Method']
df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
methods_to_plot_hm = methods_to_plot.copy()
methods_to_plot_hm.remove('Seurat')
df_dataset  = df_dataset.loc[methods_to_plot_hm,:]
df_dataset.loc[:,columns_to_scale] =scale(df_dataset.loc[:,columns_to_scale])
df_dataset = calculate_composite(df_dataset)
display(df_dataset)
plot_composite(df_dataset,save_folder+dataset)

In [ ]:
dataset = "Human_Mouse"
df = pd.read_excel(excel_path,sheet_name =sheet_name)
df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
df_dataset.index = df_dataset['Method']
df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
methods_to_plot_hm = methods_to_plot.copy()
methods_to_plot_hm.remove('Seurat')
df_dataset  = df_dataset.loc[methods_to_plot_hm,:]
df_dataset.loc[:,columns_to_scale] =scale(df_dataset.loc[:,columns_to_scale])
df_dataset = calculate_composite(df_dataset)
df_dataset = df_dataset[["Composite bio-conservation score", "Composite batch-correction score",
                         "Composite isolated label score", "Combined composite score"]]
df_dataset.to_csv('./Composite_Scores/'+dataset+'_composite_scores.csv')

# Lung

In [ ]:
dataset = "Lung"
df = pd.read_excel(excel_path,sheet_name =sheet_name)
df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
df_dataset.index = df_dataset['Method']
df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
df_dataset.loc[:,columns_to_scale] =scale(df_dataset.loc[:,columns_to_scale])
df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
df_dataset  = df_dataset.loc[methods_to_plot,:]
df_dataset = calculate_composite(df_dataset)
display(df_dataset)
plot_composite(df_dataset,save_folder+dataset)

In [ ]:
dataset = "Lung"
df = pd.read_excel(excel_path,sheet_name =sheet_name)
df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
df_dataset.index = df_dataset['Method']
df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
df_dataset.loc[:,columns_to_scale] =scale(df_dataset.loc[:,columns_to_scale])
df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
df_dataset  = df_dataset.loc[methods_to_plot,:]
df_dataset = calculate_composite(df_dataset)
df_dataset = df_dataset[["Composite bio-conservation score", "Composite batch-correction score",
                         "Composite isolated label score", "Combined composite score"]]
df_dataset.to_csv('./Composite_Scores/'+dataset+'_composite_scores.csv')

# Immune_Human

In [ ]:
dataset = "Immune_Human"
df = pd.read_excel(excel_path,sheet_name =sheet_name)
df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
df_dataset.index = df_dataset['Method']
df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
df_dataset.loc[:,columns_to_scale] =scale(df_dataset.loc[:,columns_to_scale])
df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
df_dataset  = df_dataset.loc[methods_to_plot,:]
df_dataset = calculate_composite(df_dataset)
display(df_dataset)
plot_composite(df_dataset,save_folder+dataset)

In [ ]:
dataset = "Immune_Human"
df = pd.read_excel(excel_path,sheet_name =sheet_name)
df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
df_dataset.index = df_dataset['Method']
df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
df_dataset.loc[:,columns_to_scale] =scale(df_dataset.loc[:,columns_to_scale])
df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
df_dataset  = df_dataset.loc[methods_to_plot,:]
df_dataset = calculate_composite(df_dataset)
df_dataset = df_dataset[["Composite bio-conservation score", "Composite batch-correction score",
                         "Composite isolated label score", "Combined composite score"]]
df_dataset.to_csv('./Composite_Scores/'+dataset+'_composite_scores.csv')

# composite score ablation studies

In [ ]:
datasets = ["Human_Mouse","Lung","Pancreas","Immune_Human","Human_Retina",]
sheet_name = 'Ablation'
save_folder = './Ablation/'
methods_to_plot = ['scDREAMER','scDREAMER-woDis','scDREAMER-woBC']
df = pd.read_excel(excel_path,sheet_name =sheet_name)
# display(df)
df = df.loc[df['Percentage_wrong']==0]
methods_to_scale_accross = set(df['Method'].unique()) - set(['scANVI','scGEN','scDREAMER-Sup','fastmnn'])
for dataset in datasets:
    df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
    df_dataset.index = df_dataset['Method']
    df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
    df_dataset.loc[methods_to_scale_accross,columns_to_scale] =scale(df_dataset.loc[methods_to_scale_accross,columns_to_scale])
    df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
    df_dataset  = df_dataset.loc[methods_to_plot,:]
    df_dataset = calculate_composite(df_dataset)
    df_dataset = df_dataset.loc[:,['Composite bio-conservation score','Composite batch-correction score','Composite isolated label score','Combined composite score']]
    print(dataset)
    display(df_dataset)

In [ ]:
# scVI - cyan
# Harmony - orange
# Seurat - magenta
# BBKNN - purple
# Scanorama - yellow
# INSCT - blue
# iMAP - brown
# Liger - light green
# fastMNN - light pink
# scANVI - removed from main figure
# scDREAMER - Green
# scDREAMER++ - Red

# scale with mini

In [ ]:
# def scale_with_mini(df,min_values):
#     for col in df.columns:
#         scaler = MinMaxScaler(feature_range=(min_values[col],1))
#         df.loc[:,col] = scaler.fit_transform(np.array(df.loc[:,col]).reshape(-1,1))
#     return df
# df = pd.read_excel(excel_path,sheet_name =sheet_name)
# df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
# df_dataset.index = df_dataset['Method']
# df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
# mini_values = df_dataset.loc[:,columns_to_scale].min(axis = 0)
# df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
# df_dataset = df_dataset.loc[methods_to_plot,:]
# df_dataset.loc[:,columns_to_scale] =scale_with_mini(df_dataset.loc[:,columns_to_scale],mini_values)
# df_dataset

# scale normal or existing

In [ ]:
# df = pd.read_excel(excel_path,sheet_name =sheet_name)
# df_dataset = df[df['Dataset'] == dataset].reset_index(drop = True)
# df_dataset.index = df_dataset['Method']
# df_dataset.drop(['Dataset','Method'], inplace=True, axis=1)
# df_dataset = df_dataset.loc[df_dataset['Percentage_wrong']==0]
# df_dataset = df_dataset.loc[methods_to_plot,:]
# df_dataset.loc[:,columns_to_scale] =scale(df_dataset.loc[:,columns_to_scale]
# df_dataset)